# Rainfall Prediction Model Using Random Forest Algorithm

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import pickle

In [5]:
data = pd.read_csv("../dataset/Rainfall.csv")
data.head(10)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
0,1,1025.9,19.9,18.3,16.8,13.1,72,49,yes,9.3,80.0,26.3
1,2,1022.0,21.7,18.9,17.2,15.6,81,83,yes,0.6,50.0,15.3
2,3,1019.7,20.3,19.3,18.0,18.4,95,91,yes,0.0,40.0,14.2
3,4,1018.9,22.3,20.6,19.1,18.8,90,88,yes,1.0,50.0,16.9
4,5,1015.9,21.3,20.7,20.2,19.9,95,81,yes,0.0,40.0,13.7
5,6,1018.8,24.3,20.9,19.2,18.0,84,51,yes,7.7,20.0,14.5
6,7,1021.8,21.4,18.8,17.0,15.0,79,56,no,3.4,30.0,21.5
7,8,1020.8,21.0,18.4,16.5,14.4,78,28,no,7.7,60.0,14.3
8,9,1020.6,18.9,18.1,17.1,14.3,78,79,no,3.3,70.0,39.3
9,10,1017.5,18.5,18.0,17.2,15.5,85,91,yes,0.0,70.0,37.7


In [6]:
data.shape

(366, 12)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   day                     366 non-null    int64  
 1   pressure                366 non-null    float64
 2   maxtemp                 366 non-null    float64
 3   temparature             366 non-null    float64
 4   mintemp                 366 non-null    float64
 5   dewpoint                366 non-null    float64
 6   humidity                366 non-null    int64  
 7   cloud                   366 non-null    int64  
 8   rainfall                366 non-null    object 
 9   sunshine                366 non-null    float64
 10           winddirection  365 non-null    float64
 11  windspeed               365 non-null    float64
dtypes: float64(8), int64(3), object(1)
memory usage: 34.4+ KB


In [8]:
data.duplicated().sum()

np.int64(0)

In [9]:
data.isnull().sum()

day                       0
pressure                  0
maxtemp                   0
temparature               0
mintemp                   0
dewpoint                  0
humidity                  0
cloud                     0
rainfall                  0
sunshine                  0
         winddirection    1
windspeed                 1
dtype: int64

In [11]:
# Assuming df is your DataFrame
data.columns = data.columns.str.strip()

In [12]:
data.isnull().sum()

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
rainfall         0
sunshine         0
winddirection    1
windspeed        1
dtype: int64

In [ ]:
data["rainfall"].value_counts()

rainfall
yes    249
no     117
Name: count, dtype: int64

In [14]:
percentages = data['rainfall'].value_counts(normalize=True) * 100
print(percentages)

rainfall
yes    68.032787
no     31.967213
Name: proportion, dtype: float64


In [15]:
data.sample(10)

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
131,11,1008.6,28.8,25.5,23.4,20.6,75,62,no,8.8,80.0,26.0
138,18,1012.0,26.3,24.5,23.4,19.9,76,79,no,4.8,70.0,30.3
78,19,1013.0,24.9,22.4,20.3,21.4,94,85,yes,1.5,40.0,12.5
36,6,1024.9,17.4,13.6,11.2,0.3,39,13,no,10.2,20.0,44.7
196,15,1007.0,33.0,30.2,28.6,26.4,81,78,yes,6.2,230.0,22.4
199,18,1007.5,32.4,30.4,28.7,25.3,74,72,yes,10.1,230.0,23.7
232,20,1006.3,31.2,29.1,26.8,26.1,84,67,yes,5.0,270.0,6.9
49,19,1020.3,16.3,15.3,14.6,13.3,88,94,yes,0.0,40.0,22.2
267,24,1006.7,32.0,29.3,27.4,24.7,77,23,no,11.1,230.0,17.3
261,18,1015.0,31.8,28.5,26.8,24.0,77,47,yes,8.6,70.0,14.5
